In [1]:
from harmony import BBox, Client, Collection, Request, LinkType
from harmony.config import Environment
import requests
from pprint import pprint
import datetime as dt
import s3fs
import xarray as xr

In [2]:
url = 'https://cmr.earthdata.nasa.gov/search'

In [3]:
collection_url = f'{url}/{"collections"}'

In [4]:
short_name= 'JASON_CS_S6A_L2_ALT_LR_STD_OST_NRT_F'
concept_id = 'C1968979597-POCLOUD'

In [5]:
response = requests.get(collection_url, 
                        params={
                            'concept_id': concept_id,
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
response = response.json()

In [6]:
pprint(response)

{'feed': {'entry': [{'archive_center': 'NASA/JPL/PODAAC',
                     'associations': {'services': ['S2004184019-POCLOUD',
                                                   'S1962070864-POCLOUD'],
                                      'tools': ['TL2108419875-POCLOUD',
                                                'TL2092786348-POCLOUD'],
                                      'variables': ['V2054037566-POCLOUD',
                                                    'V2054036644-POCLOUD',
                                                    'V2054037378-POCLOUD',
                                                    'V2054035071-POCLOUD',
                                                    'V2054035994-POCLOUD',
                                                    'V2054037392-POCLOUD',
                                                    'V2054037032-POCLOUD',
                                                    'V2112028269-POCLOUD',
                                                

In [7]:
services = response['feed']['entry'][0]['associations']['services']
print(services)

['S2004184019-POCLOUD', 'S1962070864-POCLOUD']


In [8]:
service_url = "https://cmr.earthdata.nasa.gov/search/services"

In [9]:
service_response = requests.get(service_url, 
                        params={
                            'concept_id': services[0],
                            },
                        headers={
                            'Accept': 'application/vnd.nasa.cmr.umm_results+json'
                            }
                       )
service_response = service_response.json()

In [10]:
pprint(service_response)

{'hits': 1,
 'items': [{'meta': {'concept-id': 'S2004184019-POCLOUD',
                     'concept-type': 'service',
                     'deleted': False,
                     'format': 'application/vnd.nasa.cmr.umm+json',
                     'native-id': 'mmt_service_14199',
                     'provider-id': 'POCLOUD',
                     'revision-date': '2021-10-13T01:56:55.795Z',
                     'revision-id': 5,
                     'user-id': 'mgangl'},
            'umm': {'Description': 'Earthdata OPEnDAP in the cloud',
                    'LongName': 'PO.DAAC OPeNDADP In the Cloud',
                    'MetadataSpecification': {'Name': 'UMM-S',
                                              'URL': 'https://cdn.earthdata.nasa.gov/umm/service/v1.4',
                                              'Version': '1.4'},
                    'Name': 'PO.DAAC Cloud OPeNDAP',
                    'OperationMetadata': [{'DistributedComputingPlatform': ['WEBSERVICES']}],
            

In [11]:
variables = response['feed']['entry'][0]['associations']['variables']
print(variables)

['V2054037566-POCLOUD', 'V2054036644-POCLOUD', 'V2054037378-POCLOUD', 'V2054035071-POCLOUD', 'V2054035994-POCLOUD', 'V2054037392-POCLOUD', 'V2054037032-POCLOUD', 'V2112028269-POCLOUD', 'V2054035154-POCLOUD', 'V2054037717-POCLOUD', 'V2054034968-POCLOUD', 'V2054035120-POCLOUD', 'V2054035569-POCLOUD', 'V2112028169-POCLOUD', 'V2054034886-POCLOUD', 'V2054036470-POCLOUD', 'V2054037696-POCLOUD', 'V2054036574-POCLOUD', 'V2054034961-POCLOUD', 'V2054036668-POCLOUD', 'V2054035135-POCLOUD', 'V2054036353-POCLOUD', 'V2054037352-POCLOUD', 'V2054035297-POCLOUD', 'V2054035663-POCLOUD', 'V2054036549-POCLOUD', 'V2054035334-POCLOUD', 'V2054036805-POCLOUD', 'V2054035487-POCLOUD', 'V2054034833-POCLOUD', 'V2054036022-POCLOUD', 'V2054037144-POCLOUD', 'V2054036031-POCLOUD', 'V2054037079-POCLOUD', 'V2054036634-POCLOUD', 'V2054034992-POCLOUD', 'V2054034812-POCLOUD', 'V2054034820-POCLOUD', 'V2054037291-POCLOUD', 'V2054037666-POCLOUD', 'V2054036133-POCLOUD', 'V2054034983-POCLOUD', 'V2054036596-POCLOUD', 'V20540365

In [12]:
var_url = "https://cmr.earthdata.nasa.gov/search/variables"

In [13]:
var_response = requests.get(var_url, 
                        params={
                            'concept_id': variables[0],
                            },
                        headers={
                            'Accept': 'application/vnd.nasa.cmr.umm_results+json'
                            }
                       )
var_response = var_response.json()

In [14]:
pprint(var_response)

{'hits': 1,
 'items': [{'associations': {'collections': [{'concept-id': 'C1968979597-POCLOUD'}]},
            'meta': {'concept-id': 'V2054037566-POCLOUD',
                     'concept-type': 'variable',
                     'deleted': False,
                     'format': 'application/vnd.nasa.cmr.umm+json',
                     'native-id': 'JASON_CS_S6A_L2_ALT_LR_STD_OST_NRT_F-data_20_ku_range_ocean_mle3',
                     'provider-id': 'POCLOUD',
                     'revision-date': '2021-10-19T03:03:05.990Z',
                     'revision-id': 2,
                     'user-id': 'jmcnelis'},
            'umm': {'DataType': 'int',
                    'Definition': 'Range computed from the MLE3 ocean '
                                  'retracker. It includes all instrumental '
                                  'corrections from '
                                  'net_instr_cor_range_ocean_mle3',
                    'Dimensions': [{'Name': 'time',
                           

In [15]:
var_list = []
for i in range(len(variables)):
    var_response = requests.get(var_url, 
                            params={
                                'concept_id': variables[i],
                                },
                            headers={
                                'Accept': 'application/vnd.nasa.cmr.umm_results+json'
                                }
                           )
    var_response = var_response.json()
    var_list.append(var_response['items'][0]['umm']['Name'])

In [16]:
pprint(var_list)

['/data_20/ku/range_ocean_mle3',
 '/data_20/c/num_iterations_ocean',
 '/data_20/ku/noise_floor_ocean',
 '/data_01/ku/net_instr_cor_sig0_ocean_mle3',
 '/data_01/orbit_type_flag',
 '/data_20/ku/num_iterations_ocean',
 '/data_20/ku/index_1hz_measurement',
 '/data_20/ku/latitude',
 '/data_01/ku/off_nadir_angle_wf_ocean',
 '/data_20/ku/surface_classification_flag',
 '/data_01/ku/model_instr_cor_range_ocean',
 '/data_01/ku/numtotal_20hz_measurement',
 '/data_01/altitude_rate',
 '/data_20/c/latitude',
 '/data_01/c/sig0_ocean_rms',
 '/data_20/c/altitude_rate',
 '/data_20/ku/ssha',
 '/data_20/c/l1b_record_counter',
 '/data_01/ku/model_instr_cor_off_nadir_angle_wf_ocean',
 '/data_20/c/range_cor_doppler',
 '/data_01/ku/ocean_geo_corrections',
 '/data_01/rain_attenuation',
 '/data_20/ku/net_instr_cor_swh_ocean',
 '/data_01/ku/range_ocean_numval',
 '/data_01/iono_cor_alt_filtered',
 '/data_20/c/epoch_ocog',
 '/data_01/ku/sea_state_bias',
 '/data_20/c/swh_ocean_qual',
 '/data_01/ku/swh_ocean_mle3',
